In [1]:
"""
WGAN-GP Training and Evaluation Notebook

This notebook demonstrates the training and evaluation of WGAN-GP for synthetic data generation.
The implementation focuses on the WGAN-GP model, which showed superior performance over
VAE and WAE in preliminary testing.
"""

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import local modules
from src.models.data_augmentation.GAN import train_and_generate
from src.utils.preprocessing import process
from src.utils.evaluation import (
    compare_statistics,
    compare_distributions,
    generate_tsne,
    recenter_data
)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configuration
DATA_PATH = "data/data_combined_controls.csv"
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

# Training parameters
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001
N_SPLITS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Configuration:")
print(f"Data path: {DATA_PATH}")
print(f"Device: {DEVICE}")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Number of folds: {N_SPLITS}")

# Data Loading and Initial Processing
print("\nLoading and processing data...")
_, _, scaled_data, scaler, n_features = process(DATA_PATH)

print("\nDataset Information:")
print(f"Original data shape: {scaled_data.shape}")
print(f"Number of features: {n_features}")
print("\nFeature types:")
print(scaled_data.dtypes)

# Generate Synthetic Data
print("\nTraining WGAN-GP and generating synthetic samples...")
synthetic_data, original_data = train_and_generate(
    filepath=DATA_PATH,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    device=DEVICE,
    n_splits=N_SPLITS,
    learning_rate=LEARNING_RATE,
    save_info=True
)

# Evaluate Original vs Synthetic Data
print("\nEvaluating synthetic data quality...")

# 1. Statistical Comparison
stats_comparison = compare_statistics(original_data, synthetic_data)
print("\nStatistical Comparison Summary:")
print(stats_comparison.describe())

# Save detailed statistics
stats_file = RESULTS_DIR / "statistical_comparison.csv"
stats_comparison.to_csv(stats_file)
print(f"\nDetailed statistics saved to: {stats_file}")

# 2. Distribution Comparison (KS Test)
ks_results = compare_distributions(original_data, synthetic_data)

# Plot KS Statistics
plt.figure(figsize=(10, 6))
sns.histplot(data=ks_results, x='KS_Statistic', kde=True)
plt.title('Distribution of KS Statistics: Original vs Synthetic Data')
plt.xlabel('KS Statistic')
plt.ylabel('Frequency')
plt.grid(True)
plt.savefig(RESULTS_DIR / "ks_statistics_distribution.png")
plt.show()

# 3. Feature-wise Visualization
print("\nGenerating feature-wise visualizations...")
for column in original_data.columns:
    if column not in ['data_type', 'fold', 'sample_id']:
        plt.figure(figsize=(10, 6))
        
        # Original data distribution
        sns.kdeplot(data=original_data[column], label='Original', alpha=0.6)
        # Synthetic data distribution
        sns.kdeplot(data=synthetic_data[column], label='Synthetic', alpha=0.6)
        
        plt.title(f'Distribution Comparison: {column}')
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True)
        plt.savefig(RESULTS_DIR / f"feature_distribution_{column}.png")
        plt.close()

# 4. t-SNE Visualization
print("\nGenerating t-SNE visualization...")
generate_tsne(original_data, synthetic_data)
plt.savefig(RESULTS_DIR / "tsne_visualization.png")
plt.show()

# 5. Summary Statistics
print("\nSummary Statistics:")
print("\nOriginal Data:")
print(original_data.describe())
print("\nSynthetic Data:")
print(synthetic_data.describe())

# 6. Quality Metrics
print("\nQuality Metrics:")
print(f"Total features with KS statistic < 0.1: {(ks_results['KS_Statistic'] < 0.1).sum()}")
print(f"Percentage of well-matched features: {(ks_results['KS_Statistic'] < 0.1).mean()*100:.2f}%")

# Save results
results_summary = {
    "n_original_samples": len(original_data),
    "n_synthetic_samples": len(synthetic_data),
    "n_features": n_features,
    "n_folds": N_SPLITS,
    "mean_ks_statistic": ks_results['KS_Statistic'].mean(),
    "percent_good_features": (ks_results['KS_Statistic'] < 0.1).mean()*100
}

with open(RESULTS_DIR / "results_summary.txt", "w", encoding='utf-8') as f:
    for key, value in results_summary.items():
        f.write(f"{key}: {value}\n")

print("\nAnalysis complete. Results saved in:", RESULTS_DIR)

Configuration:
Data path: data/data_combined_controls.csv
Device: cpu
Epochs: 20
Batch size: 32
Learning rate: 0.001
Number of folds: 5

Loading and processing data...

Dataset Information:
Original data shape: (23, 8063)
Number of features: 8063

Feature types:
IGKV2.28     float64
IGKV3D.20    float64
IGKV1.12     float64
IGLC7        float64
IGKV2.29     float64
              ...   
ZSCAN32      float64
ZSWIM8       float64
ZW10         float64
ZWILCH       float64
ZWINT        float64
Length: 8063, dtype: object

Training WGAN-GP and generating synthetic samples...

Data Overview:
Original samples: 23
Features: 8063

Feature types:
IGKV2.28     float64
IGKV3D.20    float64
IGKV1.12     float64
IGLC7        float64
IGKV2.29     float64
              ...   
ZSCAN32      float64
ZSWIM8       float64
ZW10         float64
ZWILCH       float64
ZWINT        float64
Length: 8063, dtype: object

Using device: cpu

Processing fold 1/5
Training samples: 18
Validation samples: 5
Train Loader L

Training WGAN-GP:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch [0/20] Batch [0] G_loss: 22.2386 C_loss: -121.8678


Training WGAN-GP:   5%|▌         | 1/20 [02:13<42:20, 133.72s/it]


Epoch 0 Summary:
Average G_loss: 16.3637
Average C_loss: -318.0577
Epoch [1/20] Batch [0] G_loss: -1.6380 C_loss: -39.2811


...


Epoch 1 Summary:
Average G_loss: 33.0520
Average C_loss: -394.7652
Epoch [2/20] Batch [0] G_loss: 7.2107 C_loss: -97.1136


Training WGAN-GP:  15%|█▌        | 3/20 [06:42<38:00, 134.12s/it]


Epoch 2 Summary:
Average G_loss: 18.8454
Average C_loss: -434.7357
Epoch [3/20] Batch [0] G_loss: 0.1051 C_loss: 2.3643


Training WGAN-GP:  20%|██        | 4/20 [08:56<35:45, 134.08s/it]


Epoch 3 Summary:
Average G_loss: 28.0036
Average C_loss: -390.5912
